# INTELLIGENT EMAIL AGENT - Automated Customer Support System

    WHAT THIS CODE DOES:
    Creates an AI-powered customer support agent that:
    1. Reads incoming emails
    2. Classifies intent and urgency
    3. Searches documentation/creates bug tickets as needed
    4. Drafts responses
    5. Routes critical emails to humans for review
    6. Sends approved replies
    
    THE BIG PICTURE:
    ┌──────────────────────────────────────────────────────────────────────────┐
    │                         EMAIL PROCESSING PIPELINE                        │
    ├──────────────────────────────────────────────────────────────────────────┤
    │                                                                          │
    │   Incoming Email                                                         │
    │       │                                                                  │
    │       ▼                                                                  │
    │   Classify (Intent + Urgency)                                            │
    │       │                                                                  │
    │       ├────► Billing/Critical? ──►  Human Review                         │
    │       │                                                                  │
    │       ├────► Question/Feature? ──►  Search Docs                          │
    │       │                                                                  │
    │       └────► Bug? ──────────────►  Create Ticket                         │
    │                                                                          │
    │   Draft Response                                                         │
    │       │                                                                  │
    │       ├────► High Priority? ───►  Human Review                           │
    │       │                                                                  │
    │       └────► Low Priority? ────►  Auto-Send                              │
    │                                                                          │
    └──────────────────────────────────────────────────────────────────────────┘
    
    KEY FEATURES:
    - Structured output (EmailClassification)
    - Dynamic routing based on classification
    - Human-in-the-loop for critical issues
    - Retry policies for transient failures
    - State persistence with checkpointing
    - Command pattern for explicit control flow

## PART 1: Define Data Structures (The "Schema")

In [26]:
from typing import TypedDict, Literal

    Structured output from the LLM's classification step.
    
    This ensures the AI returns data in a predictable format.
    
    Attributes:
        intent: What the customer wants
                - "question": Needs information
                - "bug": Reporting a problem
                - "billing": Payment/subscription issues
                - "feature": Requesting new functionality
                - "complex": Needs human attention
        
        urgency: How quickly we need to respond
                 - "low": Can wait days
                 - "medium": Respond within hours
                 - "high": Respond within 1 hour
                 - "critical": Immediate response needed
        
        topic: Short description (e.g., "password reset", "refund request")
        summary: One-sentence summary of the email
    
    Example:
        {
            "intent": "billing",
            "urgency": "critical",
            "topic": "double charge",
            "summary": "Customer was charged twice for subscription"
        }

In [27]:
class EmailClassification(TypedDict):
    intent: Literal["question", "bug", "billing", "feature", "complex"]
    urgency: Literal["low", "medium", "high", "critical"]
    topic: str
    summary: str

    The COMPLETE STATE that flows through the agent graph.
    
    Think of this as the agent's "working memory" - it tracks:
    - What email we're processing
    - What we've learned about it
    - What we've generated
    - Where we are in the process
    
    State Evolution Example:
    
    Step 1 (Initial):
    {
        "email_content": "I was charged twice!",
        "sender_email": "customer@example.com",
        "email_id": "email_123",
        "classification": None,
        "search_results": None,
        "customer_history": None,
        "draft_response": None,
        "messages": []
    }
    
    Step 2 (After Classification):
    {
        "email_content": "I was charged twice!",
        "sender_email": "customer@example.com",
        "email_id": "email_123",
        "classification": {
            "intent": "billing",
            "urgency": "critical",
            "topic": "double charge",
            "summary": "Customer charged twice"
        },  ← ADDED
        "search_results": None,
        "customer_history": None,
        "draft_response": None,
        "messages": [...]
    }
    
    Step 3 (After Draft):
    {
        "email_content": "I was charged twice!",
        "sender_email": "customer@example.com",
        "email_id": "email_123",
        "classification": {...},
        "search_results": None,
        "customer_history": None,
        "draft_response": "We sincerely apologize...",  ← ADDED
        "messages": [...]
    }

In [28]:
class EmailAgentState(TypedDict):
    # ─── Raw Email Data ───
    email_content: str  # The actual email text
    sender_email: str   # Who sent it
    email_id: str       # Unique identifier
    
    # ─── Classification Result ───
    classification: EmailClassification | None  # What kind of email is this?
    
    # ─── External Data ───
    search_results: list[str] | None     # Docs from knowledge base
    customer_history: dict | None        # CRM data about this customer
    
    # ─── Generated Content ───
    draft_response: str | None           # AI-drafted reply
    messages: list[str] | None           # Conversation history

## PART 2: Advanced LangGraph Concepts

### CONCEPT 1: Retry Policies

    Sometimes nodes fail due to transient issues (network timeout, API rate limit).
    RetryPolicy automatically retries failed nodes.
    
    Visual:
        ┌─────────────────────┐
        │ search_documentation│ ─── Attempt 1: Timeout 
        └─────────────────────┘
                 │
                 ▼ (wait 1 second)
        ┌─────────────────────┐
        │ search_documentation│ ─── Attempt 2: Timeout 
        └─────────────────────┘
                 │
                 ▼ (wait 2 seconds)
        ┌─────────────────────┐
        │ search_documentation│ ─── Attempt 3: Success 
        └─────────────────────┘

In [29]:
from langgraph.types import RetryPolicy

# Example: Add retry policy to a node that might fail
# workflow.add_node(
#     "search_documentation",
#     search_documentation,
#     retry_policy=RetryPolicy(
#         max_attempts=3,        # Try up to 3 times
#         initial_interval=1.0   # Wait 1 second between retries
#     )
# )

### CONCEPT 2: Command Pattern

    The Command pattern gives you EXPLICIT CONTROL over:
    1. What data to update in state
    2. Which node to go to next
    
    This replaces implicit routing with explicit decisions.
    
    Old Way (implicit):
        def my_node(state):
            return {"some_data": "value"}  # Where does it go next? 
    
    New Way (explicit):
        def my_node(state) -> Command[Literal["next_node_a", "next_node_b"]]:
            if condition:
                return Command(update={"data": "value"}, goto="next_node_a")
            else:
                return Command(update={"data": "value"}, goto="next_node_b")
    
    Visual Flow with Command:
        ┌──────────────┐
        │  execute_tool│
        └──────┬───────┘
               │
               ├──► Success? ──► Command(goto="agent")
               │
               └──► Error? ────► Command(goto="agent", with error message)

In [30]:
from langgraph.types import Command

# Example: Handle tool execution with error recovery
def execute_tool(state) -> Command[Literal["agent", "execute_tool"]]:
    """
    Example showing Command pattern with error handling.
    
    This isn't used in our email agent, but demonstrates the concept.
    """
    try:
        # Try to run the tool
        result = run_tool(state['tool_call'])
        
        # Success! Update state and go to agent
        return Command(
            update={"tool_result": result},
            goto="agent"
        )
    except ToolError as e:
        # Error! Send error message to agent so it can retry
        return Command(
            update={"tool_result": f"Tool error: {str(e)}"},
            goto="agent"  # Let LLM see the error and decide what to do
        )

### CONCEPT 3: Human-in-the-Loop with interrupt()

    interrupt() PAUSES the graph and waits for human input.
    
    Flow:
        1. Agent processes email
        2. Reaches human_review node
        3. interrupt() is called → graph PAUSES 
        4. Human reviews and approves/edits
        5. Resume execution with human's decision
    
    Visual:
        ┌───────────────┐
        │ draft_response│
        └──────┬────────┘
               │
               ▼
        ┌──────────────────────┐
        │   human_review       │
        │   interrupt()        │ ◄─── PAUSED HERE
        └──────────────────────┘
               ⋮  (waiting for human)
               ⋮
        Human provides input ✓
               ⋮
               ▼
        ┌──────────────┐
        │  send_reply  │
        └──────────────┘
    
    CRITICAL: Code before interrupt() will RE-RUN when resuming!

In [31]:
from langgraph.types import interrupt

# Example: Request customer ID from human if missing
def lookup_customer_history(state) -> Command[Literal["draft_response", "lookup_customer_history"]]:
    """
    Example showing interrupt() for requesting missing information.
    
    This isn't used in our email agent, but demonstrates the concept.
    """
    if not state.get('customer_id'):
        # Pause and ask human for customer ID
        user_input = interrupt({
            "message": "Customer ID needed",
            "request": "Please provide the customer's account ID to look up their subscription history"
        })
        
        # When resumed, update state with provided ID
        return Command(
            update={"customer_id": user_input['customer_id']},
            goto="lookup_customer_history"  # Try again with the ID
        )
    
    # Now we have the ID, proceed with lookup
    customer_data = fetch_customer_history(state['customer_id'])
    return Command(
        update={"customer_history": customer_data},
        goto="draft_response"
    )

## PART 3: Node Definitions (The Processing Pipeline)

In [32]:
from typing import Literal
from langgraph.graph import StateGraph, START, END
from langgraph.types import interrupt, Command, RetryPolicy
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage

llm = init_chat_model(
    model="qwen3:8b",
    model_provider="ollama",
    temperature=0  # 0 = deterministic, no randomness
)

### NODE 1: Read Email

    Entry point: Extract and parse email content.
    
    In production, this would:
    - Connect to email service (Gmail, Outlook, etc.)
    - Parse email headers
    - Extract attachments
    - Clean up formatting
    
    For now, just acknowledges the email in messages.
    
    Flow:
        Input State: {email_content: "...", sender_email: "...", ...}
        ↓
        Output: {messages: [HumanMessage("Processing email: ...")]}

In [33]:
def read_email(state: EmailAgentState) -> dict:
    return {
        "messages": [HumanMessage(content=f"Processing email: {state['email_content']}")]
    }

### NODE 2: Classify Intent (The Router)

    Uses LLM to classify the email and route it appropriately.
    
    This is the BRAIN of the system - it decides where emails go.
    
    Process:
    ┌─────────────────────────────────────────────────────────────────────┐
    │ 1. Take email content                                               │
    │ 2. Ask LLM to classify (intent, urgency, topic, summary)            │
    │ 3. Based on classification, decide which node to visit next         │
    └─────────────────────────────────────────────────────────────────────┘
    
    Routing Logic:
        Intent: billing OR Urgency: critical
            → human_review (humans handle money issues!)
        
        Intent: question OR feature
            → search_documentation (look up answer)
        
        Intent: bug
            → bug_tracking (create ticket)
        
        Otherwise
            → draft_response (generate reply directly)
    
    Args:
        state: Current email state
        
    Returns:
        Command with classification data and next node

In [34]:
def classify_intent(state: EmailAgentState) -> Command[Literal["search_documentation", "human_review", "draft_response", "bug_tracking"]]:
    # Create a version of the LLM that returns structured data
    # This ensures we get a dict matching EmailClassification
    structured_llm = llm.with_structured_output(EmailClassification)
    
    # Build the classification prompt
    classification_prompt = f"""
    Analyze this customer email and classify it:
    Email: {state['email_content']}
    From: {state['sender_email']}
    
    Provide classification including intent, urgency, topic, and summary.
    """
    
    # Get classification from LLM
    # Result is guaranteed to match EmailClassification structure
    classification = structured_llm.invoke(classification_prompt)
    
    # ─── Routing Logic ───
    # Decide which node to go to based on classification
    
    if classification['intent'] == 'billing' or classification['urgency'] == 'critical':
        # Money matters and emergencies need human attention
        goto = "human_review"
        
    elif classification['intent'] in ['question', 'feature']:
        # Questions and feature requests need documentation
        goto = "search_documentation"
        
    elif classification['intent'] == 'bug':
        # Bugs need tickets
        goto = "bug_tracking"
        
    else:
        # Everything else can be drafted directly
        goto = "draft_response"
    
    # Return Command with classification and routing decision
    return Command(
        update={"classification": classification},
        goto=goto
    )

### NODE 3: Search Documentation

    Search knowledge base for relevant information.
    
    This node:
    1. Extracts search query from classification
    2. Queries documentation/knowledge base
    3. Returns relevant chunks
    4. Handles search failures gracefully
    
    In production, this would:
    - Use vector database (Pinecone, Weaviate)
    - Semantic search with embeddings
    - Rank results by relevance
    
    Error Handling:
    - If search fails (network issue, API down), store error message
    - Don't crash the whole pipeline
    - Let draft_response handle the missing data
    
    Args:
        state: Contains classification with intent and topic
        
    Returns:
        Command with search results

In [35]:
def search_documentation(state: EmailAgentState) -> Command[Literal["draft_response"]]:
    # Build search query from classification
    classification = state.get('classification', {})
    query = f"{classification.get('intent', '')} {classification.get('topic', '')}"
    
    try:
        # In production: search_api.query(query)
        # For demo, return hardcoded results
        search_results = [
            "Reset password via Settings > Security > Change Password",
            "Password must be at least 12 characters",
            "Include uppercase, lowercase, numbers, and symbols"
        ]
    except Exception as e:
        # Graceful degradation: store error instead of crashing
        search_results = [f"Search temporarily unavailable: {str(e)}"]
    
    return Command(
        update={"search_results": search_results},
        goto="draft_response"
    )

### NODE 4: Bug Tracking

    Create bug tracking ticket for reported issues.
    
    In production, this would:
    - Create Jira/Linear/GitHub issue
    - Extract relevant details (steps to reproduce, environment)
    - Assign to appropriate team
    - Set priority based on urgency
    
    Args:
        state: Contains email content and classification
        
    Returns:
        Command with ticket information

In [36]:
def bug_tracking(state: EmailAgentState) -> Command[Literal["draft_response"]]:
    # In production: jira_api.create_issue(...)
    ticket_id = "BUG-12345"
    
    return Command(
        update={"search_results": [f"Bug ticket {ticket_id} created"]},
        goto="draft_response"
    )

### NODE 5: Draft Response (The Writer)

    Generate email response using all available context.
    
    This node:
    1. Gathers all context (classification, search results, customer history)
    2. Formats it into a prompt
    3. Asks LLM to draft a response
    4. Decides if human review is needed
    
    Context Assembly:
    ┌────────────────────────────────────────────────┐
    │ Classification: {intent, urgency, topic}       │
    │ Search Results: [doc1, doc2, doc3]             │
    │ Customer History: {tier, subscription, ...}    │
    └─────────────────┬──────────────────────────────┘
                      │
                      ▼
    ┌────────────────────────────────────────────────┐
    │  Formatted Prompt for LLM                      │
    └─────────────────┬──────────────────────────────┘
                      │
                      ▼
    ┌────────────────────────────────────────────────┐
    │  LLM Generates Draft Response                  │
    └────────────────────────────────────────────────┘
    
    Routing Decision:
    - High/Critical urgency → human_review
    - Complex intent → human_review  
    - Otherwise → send_reply
    
    Args:
        state: Complete state with all context
        
    Returns:
        Command with draft response and routing decision

In [37]:
def draft_response(state: EmailAgentState) -> Command[Literal["human_review", "send_reply"]]:
    classification = state.get('classification', {})
    
    # ─── Format Context Sections ───
    # Build context dynamically from available data
    context_sections = []
    
    if state.get('search_results'):
        # Format search results as bullet points
        formatted_docs = "\n".join([f"- {doc}" for doc in state['search_results']])
        context_sections.append(f"Relevant documentation:\n{formatted_docs}")
    
    if state.get('customer_history'):
        # Add customer information
        context_sections.append(
            f"Customer tier: {state['customer_history'].get('tier', 'standard')}"
        )
    
    # ─── Build the Prompt ───
    draft_prompt = f"""
    Draft a response to this customer email:
    
    {state['email_content']}
    
    Email intent: {classification.get('intent', 'unknown')}
    Urgency level: {classification.get('urgency', 'medium')}
    
    {chr(10).join(context_sections)}
    
    Guidelines:
    - Be professional and helpful
    - Address their specific concern
    - Use the provided documentation when relevant
    """
    
    # ─── Generate Response ───
    response = llm.invoke(draft_prompt)
    
    # ─── Decide if Human Review Needed ───
    needs_review = (
        classification.get('urgency') in ['high', 'critical'] or
        classification.get('intent') == 'complex'
    )
    
    goto = "human_review" if needs_review else "send_reply"
    
    return Command(
        update={"draft_response": response.content},
        goto=goto
    )

### NODE 6: Human Review (The Checkpoint)

    Pause for human review and approval.
    
    This is the HUMAN-IN-THE-LOOP checkpoint.
    
    Flow:
    1. Graph reaches this node
    2. interrupt() PAUSES execution 
    3. Human reviews the draft
    4. Human approves/edits/rejects
    5. Graph RESUMES with human's decision
    
    Critical Implementation Detail:
    ═══════════════════════════════════════════════════════════════
    interrupt() MUST come first!
    
    Any code BEFORE interrupt() will RE-RUN when resuming.
    Any code AFTER interrupt() runs only once (after human input).
    ═══════════════════════════════════════════════════════════════
    
    Visualization:
    ┌─────────────────────────────────────────────────────────────┐
    │ First Execution (up to interrupt):                          │
    │                                                             │
    │ classification = state.get('classification', {})            │
    │ human_decision = interrupt({...}) ← PAUSES HERE             │
    │                                                             │
    └─────────────────────────────────────────────────────────────┘
              ⋮
              ⋮ (waiting for human)
              ⋮
    ┌─────────────────────────────────────────────────────────────┐
    │ Second Execution (after resume):                            │
    │                                                             │
    │ classification = state.get('classification', {})  ← RE-RUNS │
    │ human_decision = interrupt({...}) ← Returns human input     │
    │ if human_decision.get("approved"): ← Runs for first time    │
    │     ...                                                     │
    └─────────────────────────────────────────────────────────────┘
    
    Args:
        state: Contains draft response and classification
        
    Returns:
        Command based on human decision

In [38]:
def human_review(state: EmailAgentState) -> Command[Literal["send_reply", END]]:
    classification = state.get('classification', {})
    
    # CRITICAL: interrupt() MUST BE FIRST
    
    # Pause and wait for human input
    human_decision = interrupt({
        "email_id": state.get('email_id', ''),
        "original_email": state.get('email_content', ''),
        "draft_response": state.get('draft_response', ''),
        "urgency": classification.get('urgency'),
        "intent": classification.get('intent'),
        "action": "Please review and approve/edit this response"
    })
    
    # ─── Process Human Decision ───
    # This code runs AFTER human provides input
    
    if human_decision.get("approved"):
        # Human approved (possibly with edits)
        return Command(
            update={
                "draft_response": human_decision.get(
                    "edited_response", 
                    state.get('draft_response', '')
                )
            },
            goto="send_reply"
        )
    else:
        # Human rejected - they'll handle it manually
        return Command(update={}, goto=END)

### NODE 7: Send Reply (The Finisher)

    Send the approved email response.
    
    In production, this would:
    - Connect to email service API
    - Format email with proper headers
    - Track sent emails
    - Log for auditing
    
    Error Handling:
    - Let unexpected errors bubble up (they're serious)
    - Retry logic should be at infrastructure level
    
    Args:
        state: Contains final draft_response
        
    Returns:
        Empty dict (end of pipeline)

In [39]:
def send_reply(state: EmailAgentState) -> dict:
    # In production: email_service.send(to=state['sender_email'], body=state['draft_response'])
    print(f"Sending reply: {state['draft_response'][:100]}...")
    return {}

## PART 4: Build the Agent Graph

    GRAPH STRUCTURE:
    ════════════════════════════════════════════════════════════════════════════
    
                                  START
                                    │
                                    ▼
                             ┌─────────────┐
                             │ read_email  │
                             └──────┬──────┘
                                    │
                                    ▼
                        ┌───────────────────────┐
                        │   classify_intent     │
                        └───────────┬───────────┘
                                    │
                        ┌───────────┼───────────┬───────────┐
                        │           │           │           │
                        ▼           ▼           ▼           ▼
              ┌─────────────┐ ┌────────┐ ┌─────────┐ ┌────────────┐
              │   search    │ │  bug   │ │  draft  │ │   human    │
              │documentation│ │tracking│ │response │ │  review    │
              └──────┬──────┘ └───┬────┘ └────┬────┘ └─────┬──────┘
                     │            │           │            │
                     └────────────┴──────► draft_response  │
                                            │              │
                                       ┌────┴────┐         │
                                       ▼         ▼         │
                                ┌────────────┐ ┌────────────┐
                                │   send     │ │   human    │
                                │   reply    │ │  review    │
                                └──────┬─────┘ └─────┬──────┘
                                       │             │
                                       ├─────────────┤
                                       │             │
                                       ▼             ▼
                                     END           send_reply
                                                     │
                                                     ▼
                                                   END
    
    Key Features:
    - Multiple routing paths from classify_intent
    - Conditional routing from draft_response
    - Human review can go to send_reply OR END
    - Retry policy on search_documentation

In [40]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import RetryPolicy

# Create the graph
workflow = StateGraph(EmailAgentState)

### Add Nodes

In [41]:
workflow.add_node("read_email", read_email)
workflow.add_node("classify_intent", classify_intent)

# Add retry policy for nodes that might have transient failures
workflow.add_node(
    "search_documentation",
    search_documentation,
    retry_policy=RetryPolicy(max_attempts=3)  # Retry up to 3 times
)

workflow.add_node("bug_tracking", bug_tracking)
workflow.add_node("draft_response", draft_response)
workflow.add_node("human_review", human_review)
workflow.add_node("send_reply", send_reply)

### Add Edges (Only the non-conditional ones)

In [42]:
# Fixed edges that always happen
workflow.add_edge(START, "read_email")
workflow.add_edge("read_email", "classify_intent")
workflow.add_edge("send_reply", END)

# Note: Conditional edges are handled by Command.goto in the node functions

### Compile with Checkpointer

    Checkpointer enables:
    1. State Persistence: Save state between runs
    2. Resume Capability: Continue from where you left off
    3. Time Travel: Go back to previous states
    4. Human-in-the-Loop: Essential for interrupt() to work
    
    Without checkpointer: interrupt() won't work (no way to resume)
    With checkpointer: Can pause and resume execution

In [43]:
memory = MemorySaver()  # In-memory checkpointer (use Redis/Postgres in production)
app = workflow.compile(checkpointer=memory)

## PART 5: Run the Agent

### STEP 1: Create Initial State

In [46]:
initial_state = {
    "email_content": "I was charged twice for my subscription! This is urgent!",
    "sender_email": "customer@example.com",
    "email_id": "email_123",
    "messages": []
}

# Config with thread_id for persistence
# The thread_id groups related executions together
config = {"configurable": {"thread_id": "customer_123"}}

print("\n[Initial Email]")
print(f"From: {initial_state['sender_email']}")
print(f"Content: {initial_state['email_content']}")


[Initial Email]
From: customer@example.com
Content: I was charged twice for my subscription! This is urgent!


### STEP 2: First Invocation (Will Pause at Human Review)

    Execution Trace:
    1. START → read_email
       - Acknowledges email
       
    2. read_email → classify_intent
       - LLM classifies: intent="billing", urgency="critical"
       - Routes to human_review (billing + critical = human needed!)
       
    3. classify_intent → human_review
       - Generates draft response
       - interrupt() is called → PAUSES 
       
    Graph is now WAITING for human input...

In [48]:
print("\n[Starting Agent...]")
result = app.invoke(initial_state, config)

print("\n[Agent Paused at Human Review]")
print(f"Draft ready for review: {result['draft_response']}")
print(f"Classification: {result['classification']}")


[Starting Agent...]

[Agent Paused at Human Review]
Draft ready for review: <think>
Okay, the user received a customer email about being charged twice for their subscription and it's urgent. Let me start by understanding the situation. The customer is upset and needs a quick resolution. The guidelines say to be professional, address their concern, and use documentation if needed.

First, I should acknowledge their urgency and apologize for the inconvenience. It's important to show empathy. Then, I need to ask for their subscription details to identify the charges. Maybe mention that I can check the billing history. Also, offer to refund the duplicate charge once confirmed. I should keep the tone calm and reassuring, making sure they feel supported. Let me structure the response step by step: start with a greeting, express regret, request necessary info, explain the next steps, offer a refund, and close with contact info. Make sure it's clear and concise without any jargon. Check if th

### STEP 3: Human Provides Input

In [49]:
print("\n[Human reviewing and editing draft...]")

# Simulate human approval with edits
from langgraph.types import Command

human_response = Command(
    resume={
        "approved": True,
        "edited_response": "We sincerely apologize for the double charge. I've initiated an immediate refund that should appear in your account within 3-5 business days. As a gesture of goodwill, we've also added a month of free service to your account. If you have any other concerns, please don't hesitate to reach out."
    }
)


[Human reviewing and editing draft...]


### STEP 4: Resume Execution

    Continued Execution:
    4. human_review (resumed)
       - Processes human's approval
       - Updates draft with edited version
       - Routes to send_reply
       
    5. send_reply → END
       - Sends the email
       - Workflow complete 

In [52]:
print("\n[Resuming agent with human approval...]")

# Resume with the same config (same thread_id)
final_result = app.invoke(human_response, config)

print(f"\n[Email sent successfully!]")
print(f"Final response (first 200 chars):")
print(f"{final_result.get('draft_response', '')[:200]}...")


[Resuming agent with human approval...]

[Email sent successfully!]
Final response (first 200 chars):
We sincerely apologize for the double charge. I've initiated an immediate refund that should appear in your account within 3-5 business days. As a gesture of goodwill, we've also added a month of free...
